## Librerie

In [1]:
from scipy.optimize import minimize
import numpy as np
import pandas as pd

## Parametri problema

In [2]:
# Consumi famiglia 1, famiglia 2, ...
consumi = np.array([1500, 5700, 3400, 300])

# Produzioni ricetta1, ricetta2, ...
produzioni = np.array([3000, 6300, 1200])

# Ricette
#           | Ricetta1 | Ricetta2 | ...
# --------------------------------------
# Famiglia1 |          |          |
# Famiglia2 |          |          |
# ...
ricette = np.array([
    0.25, 0.2, 0.3,
    0.43, 0.5, 0.35,
    0.3, 0.27, 0.35,
    0.02, 0.03,  0
])

# Composizioni ricette per famiglia
#           | Materiale1 | Materiale2 | ...
# --------------------------------------
# Famiglia1 |            |            |
# Famiglia2 |            |            |
# ...
composizioni_famiglia = np.array([
    0.58, 0.42, 0,
    1, 0, 0,
    0, 1, 0,
    0, 0, 1
])

# Range ammissibile percentuale materiale per ricetta
#           | Materiale1        | Materiale2 | ...
# ------------------------------------------------
# Ricetta1  | (val att. ,range) |            |
# Ricetta2  |                   |            |
# ...
range_ric_mat = np.array([
    #[(0.58, 0.01), (0.396, 0.003), (0.024, 0.001)], # ricetta 0
    #[(0.625, 0.005), (None, None), (None, None)],
    #[(0.62, 0.01), (None, None), (None, None)],
])

In [3]:
%run ./importazione.ipynb

(10,)
(28,)
(28, 10)
7
17
56
57
(28, 13)


## Calcolo resa globale

In [4]:
tot_consumi = np.sum(consumi)
tot_produzioni = np.sum(produzioni)
resa_globale = tot_consumi / tot_produzioni
f'{resa_globale=}'

'resa_globale=np.float64(1.0389346559077537)'

## Funzioni di calcolo

In [5]:
# Calcola matrice consumi moltiplicando matrice ricetta in input per produzioni
def calc_mat_consumi(ricetta):
    return ricetta.reshape(-1, len(produzioni)) * produzioni

In [6]:
# Calcola vettore consumi complessivi partendo da produzioni iniziali e matrice consumi
def calc_tot_consumi(matrice_consumi):
    return np.sum(matrice_consumi, axis=1)

In [7]:
# Calcolo errore su totali consumi
def calc_err_totali(ricetta):
    matrice_consumi = calc_mat_consumi(ricetta)
    tot_consumi = calc_tot_consumi(matrice_consumi)
    tot_err = np.sum(np.square(tot_consumi-consumi))
    return tot_err

In [8]:
# Calcola rese per famiglia 
# (consumi per famiglia / produzione)
def calc_tot_resa(matrice_consumi):
    return np.sum(matrice_consumi, axis=0)/produzioni

In [9]:
# Calcolo errore su percentuali prod. effettive rispetto a resa totale
def calc_error_resa(ricetta):
    matrice_consumi = calc_mat_consumi(ricetta)
    tot_resa = calc_tot_resa(matrice_consumi)
    return np.sum(np.square(tot_resa - resa_globale))

## Ottimizzazione

In [20]:
%%time

constraints = [
    {'type': 'eq', 'fun': calc_err_totali},
]

bounds = list(( (0, None) for x in range(len(produzioni)*len(consumi)) ))
# bounds[4] = (0, 0.5) #DEBUG

res = minimize(
    calc_error_resa, 
    ricette, 
    method='SLSQP',
    constraints=constraints,
    bounds=bounds,
    options={'disp': True, 'maxiter':500}
)

print(res)

Iteration limit reached    (Exit mode 9)
            Current function value: 2.864162117214216e-13
            Iterations: 500
            Function evaluations: 144224
            Gradient evaluations: 500
 message: Iteration limit reached
 success: False
  status: 9
     fun: 2.864162117214216e-13
       x: [ 2.269e-02  2.510e-03 ...  5.104e-06  1.518e-02]
     nit: 500
     jac: [ 1.602e-07  7.747e-07 ...  1.685e-07 -5.964e-09]
    nfev: 144224
    njev: 500
CPU times: total: 18.5 s
Wall time: 36.3 s


In [11]:
# Esperimento
# Proviamo a escludere un constraint alla volta e vediamo se troviamo una situazione
# in cui il problema converge.

# for i in range(1, len(constraints)):
    
#     print(i)
    
#     const_red = constraints[0:i] + constraints[i+1:]

#     res = minimize(
#         calc_error_resa, 
#         ricette, 
#         method='SLSQP',
#         constraints=const_red,
#         bounds=bounds,
#         options={'disp': False, 'maxiter':100}
#     )

#     print(f'{res.success} - {res.message} ({res.nit} iterations)')


## Verifiche

In [12]:
pd.options.display.float_format = '{:.2f}'.format

In [13]:
print("Percentuali aggiustate (in %)")
pd.DataFrame(
    res.x.reshape(len(consumi), len(produzioni))*100
)

Percentuali aggiustate (in %)


,0,1,2,3,4,5,6,7,8,9
0,2.33,0.28,7.11,0.01,3.55,5.28,3.37,3.40,3.93,3.51
1,69.70,76.09,13.48,41.12,47.86,18.50,4.92,32.25,39.83,3.34
2,2.06,0.76,15.91,1.17,3.58,5.28,3.37,3.40,3.99,3.51
3,0.22,0.30,0.09,0.39,10.84,3.94,3.17,1.84,12.21,3.22
4,0.00,0.01,0.00,0.07,0.00,0.00,1.02,0.13,0.00,0.15
5,0.22,0.17,2.68,0.06,1.29,4.47,3.26,13.85,0.91,3.34
6,0.00,0.00,0.00,0.01,0.00,0.03,2.49,0.04,0.00,2.66
7,0.26,0.16,0.09,0.14,0.06,3.94,3.18,1.85,0.59,3.23
8,11.20,15.84,9.03,36.64,3.72,4.93,3.12,3.05,4.56,3.33
9,2.09,0.68,16.33,0.31,4.85,5.23,3.37,3.39,3.95,3.51


In [14]:
print("Matrice consumi")
pd.DataFrame(
    calc_mat_consumi(res.x)
)

Matrice consumi


,0,1,2,3,4,5,6,7,8,9
0,135068.91,97776.15,531600.75,10557.87,133175.30,71795.65,6779.88,53448.62,197001.99,10036.43
1,4041024.33,26625574.54,1008308.14,38346522.42,1795693.40,251302.55,9891.72,506332.65,1996759.55,9554.87
2,119216.88,266908.88,1190010.81,1094891.11,134436.10,71709.16,6760.26,53426.03,200001.81,10022.21
3,12767.02,104735.67,7092.64,366614.02,406729.48,53481.74,6367.76,28877.86,612081.81,9212.45
4,0.11,3353.35,0.00,62162.19,0.00,0.00,2047.75,2069.11,3.88,440.67
5,13007.20,58272.39,200214.63,58497.24,48387.37,60750.15,6543.67,217460.17,45469.19,9552.13
6,0.00,0.00,69.45,9415.24,19.41,378.87,5005.98,562.56,0.00,7599.31
7,15209.56,54293.33,7012.31,129856.88,2283.44,53528.76,6389.69,29027.93,29415.91,9234.92
8,649247.56,5544569.52,675695.43,34165894.16,139549.99,66942.17,6264.56,47927.34,228353.63,9508.47
9,121397.10,238397.09,1221932.74,291886.09,181961.68,71059.66,6762.97,53262.02,197930.52,10020.80


In [15]:
print('Verifica totale consumi')
pd.DataFrame(
    [calc_tot_consumi(calc_mat_consumi(res.x))]
)

Verifica totale consumi


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,1247241.54,74590964.18,3147383.26,1607960.45,70077.05,718154.15,23050.83,336252.72,41533952.83,2394610.67,...,10409876.69,1486554.90,58042.28,4510.71,5823.14,14156.89,24290.98,2722.63,6914.33,12734.92


In [16]:
print('Verifica errore consumi')
pd.DataFrame([calc_tot_consumi(calc_mat_consumi(res.x)) - consumi])

Verifica errore consumi


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,-140.46,-323.55,-544.62,313.85,-186.45,-123.15,-576.17,32.72,-106.51,-278.33,...,-330.31,327.90,160.28,2160.71,1381.14,1951.89,-36.02,2509.63,518.83,1891.92


In [17]:
print(f'Verifica rese (resa globale: {resa_globale:.2f})')
pd.DataFrame(
    [calc_tot_resa(calc_mat_consumi(res.x))]
)

Verifica rese (resa globale: 1.04)


,0,1,2,3,4,5,6,7,8,9
0,1.04,1.04,1.04,1.04,1.04,1.04,1.04,1.04,1.04,1.04


In [18]:
print(f'Verifica errore rese (resa globale: {resa_globale:.2f})')
pd.DataFrame(
    [calc_tot_resa(calc_mat_consumi(res.x)) - resa_globale]
)

Verifica errore rese (resa globale: 1.04)


,0,1,2,3,4,5,6,7,8,9
0,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00
